In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/twitter-user-gender-classification/gender-classifier-DFE-791531.csv", encoding = "latin1")
data

In [ ]:
data.columns

In [ ]:
data = pd.concat([data.gender,data.description], axis = 1) # iki tane dataframe veya series i birleştirmektir.
data

In [ ]:
data.info()

In [ ]:
data.dropna(axis = 0,inplace = True)
data.info()

In [ ]:
data.gender = [1 if each == "female" else 0 for each in data.gender]

In [ ]:
# cleaning data
# regular expression RE
import re

first_description = data.description[4]
description = re.sub("[^a-zA-Z]"," ",first_description)
description = description.lower()
description

In [ ]:
# stopwords (irrelavent words) alakasız kelimeler
import nltk # natural language tool kit
nltk.download("stopwords")
from nltk.corpus import stopwords

In [ ]:
description = nltk.word_tokenize(description)
description

In [ ]:
# split kullanılırsa shouldn't gibi kelimeler should not olarak ayrılmaz fakat word_tokenize kullanılırsa belirtildiği gibi ayrılabilir.

description = [word for word in description if not word in set(stopwords.words("english"))]
description

In [ ]:
# lemmatization
# kelimelerin köklerini bulma
import nltk as nlp

lemma = nlp.WordNetLemmatizer()
description = [lemma.lemmatize(word) for word in description ]
description

In [ ]:
description = " ".join(description)
description

In [ ]:
# tüm data için uygulama
descrip_list = []
for descrip in data.description:
    
    descrip = re.sub("[^a-zA-Z]"," ",descrip)
    descrip = descrip.lower()
    descrip = nltk.word_tokenize(descrip)
    #descrip = [word for word in descrip if not word in set(stopwords.words("english"))]
    lemma2 = nlp.WordNetLemmatizer()
    descrip = [lemma2.lemmatize(word) for word in descrip]
    descrip = " ".join(descrip)
    descrip_list.append(descrip)

In [ ]:
descrip_2 = pd.DataFrame(descrip_list)

In [ ]:
# bag of words

from sklearn.feature_extraction.text import CountVectorizer

max_features = 5000 # en çok kullanılan kelimeden 500 tanesini seç demektir.

count_vectorizer = CountVectorizer(max_features=max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(descrip_list).toarray()
print("en sik kullanilan {} kelimeler: {}".format(max_features, count_vectorizer.get_feature_names()))


In [ ]:
y = data.iloc[:,0].values # male or female classes
x = sparce_matrix
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1,random_state = 42)


In [ ]:
# naive bayes

from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(x_train,y_train)

In [ ]:
# prediction

y_predict = nb.predict(x_test)

# accuracy

print("accuracy: ", nb.score(y_predict.reshape(-1,1),y_test))